# 데이터셋 설명

- train.csv 
    - ID : 대구에서 발생한 교통사고의 고유 ID
    - 2019년부터 2021년까지의 교통사고 데이터로 구성
    - 해당 사고가 발생한 당시의 시공간 정보와 사고 관련 정보 포함 
    - ECLO : 인명피해 심각도


- test.csv
    - ID : 대구에서 발생한 교통사고의 고유 ID
    - 2022년도의 교통사고 데이터로 구성 
    - 추론 시점에서 획득할 수 있는 정보로 구성

# 문제 설명
- train 데이터셋을 활용하여 학습하고, test 데이터셋의 `ECLO`(인명 피해 심각도)를 예측합니다.

# Load dataset

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
INPUT_PATH = './input/'

train = pd.read_csv(f'{INPUT_PATH}train.csv')
test = pd.read_csv(f'{INPUT_PATH}test.csv')
ss = pd.read_csv(f'{INPUT_PATH}sample_submission.csv')

# Data validation

- 유효성 검증 함수 생성

In [3]:
def making_val_table(df):
    '''입력받은 데이터셋의 유효성 검증을 위한 요약 테이블'''
    # dtypes 
    df_dtypes = df.dtypes
    
    # nunique
    df_nunique = df.nunique()
    
    # null_values
    df_nan = df.isna().sum()
    
    val_table = pd.concat([df_dtypes, df_nunique, df_nan], axis=1)
    val_table.columns = ['dtype', 'nunique', 'nan']
        
    return val_table.reset_index()

**특이사항**

- `ID` 컬럼은 unique 값으로 학습에 불필요하므로 제거합니다.
- `사고일시` 컬럼이 object로 되어 있어 datetime으로 변환이 필요합니다.

- train과 test 데이터셋 간의 컬럼 수가 서로 다릅니다.
    - test 데이터의 컬럼만을 사용하여 train 데이터를 학습합니다.
- `기상상태`, `시군구` 컬럼의 고유값 수(nunique)가 서로 다릅니다.
    - 고유값이 서로 다른 경우 test 데이터의 예측 값 생성 시 에러가 발생할 수 있습니다. 
    - 두 컬럼의 고유값 수가 다르므로 제거합니다. 
    - 현재 노트는 베이스라인 모델을 위한 노트로, 이후 본 분석에서는 train 데이터의 나머지 컬럼들과 주어진 외부 데이터들을 이용하여 추가 분석을 진행할 예정입니다.
    

| 구분|train|test |
|---|---|---|
| 컬럼 수 |23개 | 8개 |
| 기상상태 nunique | 6개 | 5개 |
| 시군구 nunique | 199개 | 192개 |

In [4]:
# 기상상태, 시군구 데이터의 unique 값이 각각 7vs6, 199vs 192로 다르다
train_info = making_val_table(train)
test_info = making_val_table(test)

pd.merge(left=train_info, right=test_info, on='index', 
         how='left', suffixes=('_train','_test')).set_index('index')

,dtype_train,nunique_train,nan_train,dtype_test,nunique_test,nan_test
index,,,,,,
ID,object,39609,0,object,10963.0,0.0
사고일시,object,18057,0,object,5548.0,0.0
요일,object,7,0,object,7.0,0.0
기상상태,object,6,0,object,5.0,0.0
시군구,object,199,0,object,192.0,0.0
도로형태,object,11,0,object,11.0,0.0
노면상태,object,6,0,object,6.0,0.0
사고유형,object,3,0,object,3.0,0.0
사고유형 - 세부분류,object,14,0,NaN,NaN,NaN


# Feature engineering

**전처리 과정**
- train, test 데이터셋 간의 컬럼 동기화
- `사고일시` 컬럼 datetime으로 변환 후
    - `월`, `일`, `시` 컬럼생성, 년도데이터는 제거 
- `ID`,`사고일시`, `기상상태`, `시군구` 컬럼 제거
- one-hot encoding 실시

- train, test 데이터셋의 컬럼 동기화 실시

In [5]:
# target 변수 지정
y_train = train['ECLO']

# train, test 데이터간 컬럼 동기화 
cols = test.columns
X_train = train[cols]
X_test = test

- 기타 전처리함수 생성

In [6]:
def feat_eng(df):
    # datetime 변환
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    
    # 월,일,시 컬럼 생성
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['시'] = df['사고일시'].dt.hour
    
    # 불필요 컬럼 제거
    subs = ['ID','사고일시','기상상태','시군구']
    df = df.drop(subs, axis=1)
    
    # one-hot encoding 실시
    df = pd.get_dummies(df)
    
    return df

- train, test 전처리 실시

In [7]:
X_train_eng = feat_eng(X_train)
X_test_eng = feat_eng(X_test)

- 전처리 후 shape 확인

In [8]:
print(f'X_train 데이터 shape : {X_train_eng.shape}')
print(f'y_train 데이터 shape : {y_train.shape}')
print()
print(f'X_test 데이터 shape : {X_test_eng.shape}')

X_train 데이터 shape : (39609, 30)
y_train 데이터 shape : (39609,)

X_test 데이터 shape : (10963, 30)


# Modeling

- 검증을 위한 데이터 분리

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_eng, y_train , test_size=0.2, random_state=42)

- 랜덤포테스트 회귀모델 학습 진행

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
y_val_pred = rf.predict(X_val)

# Evaluation

- 문제에서 제시한 RMSLE 지표를 이용한 평가 진행

In [11]:
from sklearn.metrics import mean_squared_log_error

rmsle_train = np.sqrt(mean_squared_log_error(y_train, y_train_pred))
rmsle_val = np.sqrt(mean_squared_log_error(y_val, y_val_pred))

In [12]:
print('train rmsle :', rmsle_train)
print('val rmsle :', rmsle_val)

train rmsle : 0.2560802972254713
val rmsle : 0.5093072550464652


# Submission 

- test 데이터셋에 대한 예측값 생성

In [13]:
y_pred = rf.predict(X_test_eng)
y_pred

array([4.16      , 2.95      , 7.19      , ..., 5.73      , 7.48      ,
       6.29466667])

- 제출 파일형식으로 변환 후 제출 파일 생성

In [14]:
ss['ECLO'] = y_pred
ss.head()

,ID,ECLO
0,ACCIDENT_39609,4.160000
1,ACCIDENT_39610,2.950000
2,ACCIDENT_39611,7.190000
3,ACCIDENT_39612,7.170000
4,ACCIDENT_39613,4.946667


In [15]:
OUTPUT_PATH = './output/'
ss.to_csv(f'{OUTPUT_PATH}submission.csv', index=False)